In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest

In [ ]:
df = pd.read_csv('/kaggle/input/nba-shot-logs/shot_logs.csv')
df.head()

In [ ]:
# convert game clock to seconds sigh
df['GAME_CLOCK_S'] = (pd.to_datetime(df['GAME_CLOCK'], format='%M:%S') - pd.Timestamp('1900-01-01')).dt.total_seconds()

In [ ]:
# let's drop rows where shot clock is nan (other columns don't have nans)
df.dropna(inplace=True)

### shot prediction

#### some baselines

In [ ]:
# only use shot distance
df.groupby(['FGM'])['SHOT_DIST'].mean()

In [ ]:
# cheating a bit here by looking at full sample
max_acc, max_t = 0, 0
for t in np.arange(1, int(df['SHOT_DIST'].max()) + 1, 0.1):
    acc = (df[df['SHOT_DIST'] < t]['FGM'].sum() - (df[df['SHOT_DIST'] >= t]['FGM'] - 1).sum()) / len(df)
    if acc > max_acc:
        max_acc = acc
        max_t = t
max_t, max_acc

In [ ]:
# only use player id (again, cheating a bit by looking at full sample)
fgp = df.groupby(['player_id'])['FGM'].agg(['mean', 'count'])
fgp['absmean'] = np.abs(0.5 - fgp['mean']) + 0.5
np.average(fgp['absmean'], weights=fgp['count'])

#### the big brain stuff

In [ ]:
num = ['FINAL_MARGIN', 'GAME_CLOCK_S', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME', 'SHOT_DIST', 'CLOSE_DEF_DIST']
cat = ['LOCATION', 'W', 'SHOT_NUMBER', 'PERIOD', 'PTS_TYPE', 'CLOSEST_DEFENDER_PLAYER_ID', 'player_id']
ycol = 'FGM' # same as 'SHOT_RESULT'

In [ ]:
ct = ColumnTransformer([
#     ('ohe', OneHotEncoder(), cat), 
    ('scaler', StandardScaler(), num)
])
pipe = Pipeline([('ct', ct), ('fs', SelectKBest(k='all'))])
X, y = pipe.fit_transform(df[num+cat], y=df[ycol]), df[ycol]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=11)

In [ ]:
# let's import our favorite classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# cls = LogisticRegression(C=11, max_iter=500).fit(X_train, y_train)
cls = GradientBoostingClassifier().fit(X_train, y_train)

In [ ]:
y_valid_pred = cls.predict(X_valid)
print(classification_report(y_valid, y_valid_pred))

guess this problem is sorta hard